# Классификация текстов по функциональным стилям (RuBERT + pymorphy3)

В данном ноутбуке мы:
1. Установим и импортируем необходимые библиотеки.
2. Проведём предобработку текстов (очистка + лемматизация с помощью `pymorphy3`).
3. Используем модель [RuBERT](https://huggingface.co/DeepPavlov/rubert-base-cased) от DeepPavlov для обучения на задаче классификации текстов.
4. Оценим качество на тестовых данных.
5. Продемонстрируем применение модели к новым текстам.


## Шаг 1. Установка необходимых библиотек

> **Примечание:** Если вы используете Google Colab или свою локальную среду, где данные библиотеки уже установлены, возможно, ничего дополнительно делать не нужно. Но если возникает ошибка "No module named ...", раскомментируйте соответствующие команды.

In [ ]:
# !pip install pymorphy3 transformers datasets torch scikit-learn

## Шаг 2. Импорт и инициализация

In [2]:
import re
import numpy as np
import pymorphy3
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Инициализируем лемматизатор pymorphy3
morph = pymorphy3.MorphAnalyzer()

# Для воспроизводимости
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


## Шаг 3. Подготовка (сбор) данных
В реальном случае вы можете загружать тексты и метки (стили) из CSV/JSON/базы данных. Здесь для примера используем короткий искусственный корпус.

In [3]:
# Пример данных: тексты и их классы (5 стилей)
texts = [
    "Изучая квантовую механику, важно понимать принципы неопределённости Гейзенберга.",
    "Согласно Постановлению №123, гражданам требуется подать заявление до 1 мая.",
    "Вчера в новостях сообщили о повышении цен на топливо и провели опрос среди населения.",
    "Привет! Как дела? Давно не виделись, давай созвонимся вечером!",
    "Утрами морозное солнце красило дворец в ослепительно-пурпурные тона...",
    "В ходе эксперимента было обнаружено, что данные частиц не совпадают с теоретическими предсказаниями.",
    "Настоящим подтверждается, что указанный сотрудник состоит в штате на должности менеджера.",
    "По информации СМИ, на этой неделе состоится конференция по экологическим проблемам.",
    "Привет, ты уже в городе? Как насчёт встретиться и обсудить планы на выходные?",
    "Её глаза блестели, а сердце замирало в предчувствии неизведанных дорог."
]

# Метки (0=научный, 1=официально-деловой, 2=публицистический, 3=разговорный, 4=художественный)
labels = [
    0, 1, 2, 3, 4,
    0, 1, 2, 3, 4
]

len(texts), len(labels)

(10, 10)

## Шаг 4. Предобработка текстов (очистка + лемматизация через `pymorphy3`)

In [4]:
def preprocess_text(text: str) -> str:
    # 1) Приводим в нижний регистр
    text = text.lower()
    # 2) Убираем пунктуацию и нежелательные символы
    text = re.sub(r'[\^\!\?\.,:\-\—\"№;\(\)\"\…\«\»]', ' ', text)
    # 3) Убираем повторяющиеся пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    # 4) Лемматизируем каждое слово
    tokens = text.split()
    lemma_tokens = [morph.parse(token)[0].normal_form for token in tokens]
    lemmatized_text = ' '.join(lemma_tokens)
    return lemmatized_text

processed_texts = [preprocess_text(t) for t in texts]
processed_texts

['изучать квантовый механика важно понимать принцип неопределённость гейзенберг',
 'согласно постановление 123 гражданин требоваться подать заявление до 1 май',
 'вчера в новость сообщить о повышение цена на топливо и провести опрос среди население',
 'привет как дело давно не видеться давать созвониться вечером',
 'утро морозный солнце красить дворец в ослепительно пурпурный тон',
 'в ход эксперимент быть обнаружить что дать частица не совпадать с теоретический предсказание',
 'настоящий подтверждаться что указанный сотрудник состоять в штат на должность менеджер',
 'по информация сми на этот неделя состояться конференция по экологический проблема',
 'привет ты уже в город как насчёт встретиться и обсудить план на выходной',
 'её глаз блестеть а сердце замирать в предчувствие неизведанный дорога']

## Шаг 5. Разделение на обучающую и тестовую выборки
Используем функцию `train_test_split` из scikit-learn.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    processed_texts,
    labels,
    test_size=0.5,
    random_state=42,
    stratify=labels
)
len(X_train), len(X_test), y_train, y_test

(5, 5, [3, 1, 2, 0, 4], [1, 0, 3, 4, 2])

## Шаг 6. Создание объектов `Dataset` (Hugging Face) для обучения
Чтобы пользоваться `Trainer`, преобразуем данные в формат `Dataset`.

In [7]:
train_dataset = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})
test_dataset = Dataset.from_dict({
    'text': X_test,
    'label': y_test
})

train_dataset, test_dataset

(Dataset({
     features: ['text', 'label'],
     num_rows: 5
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 5
 }))

## Шаг 7. Загрузка токенизатора и модели RuBERT
Используем модель [DeepPavlov/rubert-base-cased](https://huggingface.co/DeepPavlov/rubert-base-cased). Зададим `num_labels=5`, чтобы выйти на классификацию по пяти стилям.

In [8]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Функция токенизации для набора данных
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Токенизируем train и test
train_dataset = train_dataset.map(tokenize_fn, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize_fn, batched=True, batch_size=len(test_dataset))

# Приводим к формату PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

## Шаг 8. Настройка `Trainer` и гиперпараметров обучения

In [10]:
training_args = TrainingArguments(
    output_dir="test_style_rubert_pymorphy3",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,              # Пример: 3 эпох
    per_device_train_batch_size=2,   # Батч для обучения
    per_device_eval_batch_size=2,    # Батч для валидации
    logging_dir="logs",
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    seed=42
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

## Шаг 9. Обучение модели

In [ ]:
trainer.train()

## Шаг 10. Оценка на тестовом наборе

In [ ]:
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", eval_results)

# Подробный отчёт
predictions = trainer.predict(test_dataset)
pred_label_ids = np.argmax(predictions.predictions, axis=-1)

target_names = [
    "Научный", 
    "Официально-деловой", 
    "Публицистический", 
    "Разговорный", 
    "Художественный"
]

print("\nClassification Report (Test):")
print(classification_report(
    y_test,
    pred_label_ids,
    target_names=target_names
))

## Шаг 11. Пример применения модели на новых текстах

In [ ]:
new_texts = [
    "Данный эксперимент доказывает существование новых возмущений в ядре атома.",
    "Настоящим уведомляем Вас о необходимости явиться в суд.",
    "Вчера по телевидению показали репортаж о ситуации с пробками на дорогах.",
    "Привет! Ты видел последний фильм? Давай обсудим!",
    "Ветер шептал о грядущих переменах, а утро встречало её ласковым рассветом."
]

# Повторяем ту же схему предобработки
new_texts_preprocessed = [preprocess_text(t) for t in new_texts]

# Токенизируем
encodings = tokenizer(
    new_texts_preprocessed,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Прогоняем через модель
with torch.no_grad():
    outputs = model(**encodings)
    logits = outputs.logits
    predicted_classes = torch.argmax(logits, dim=1).numpy()

label_map = {
    0: "Научный",
    1: "Официально-деловой",
    2: "Публицистический",
    3: "Разговорный",
    4: "Художественный"
}

print("\nНовые тексты и их стили:\n")
for text, pred_label in zip(new_texts, predicted_classes):
    print(f"Текст: {text}\n -> {label_map[pred_label]}\n")

### Заключение
Этот ноутбук аналогичен предыдущему примеру (с `pymorphy2`), но использует **pymorphy3** для лемматизации. В остальном шаги остаются теми же:
1. Очистка текста.
2. Лемматизация через `pymorphy3.MorphAnalyzer`.
3. Токенизация и обучение модели `RuBERT`.
4. Оценка и применение к новым текстам.

**Возможные улучшения**:
- Расширить датасет (для реальной задачи нужно гораздо больше примеров).
- Оптимизировать гиперпараметры (число эпох, размер батча, learning rate и т.п.).
- Проверять дополнительные метрики (precision, recall, f1) и анализировать результаты.
- Применять кросс-валидацию, аугментацию данных и т.д.